In [10]:
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import nltk
nltk.download('stopwords') 
nltk.download('wordnet')
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
import os
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Chike\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Chike\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
embeddings_index={}
f=open(os.path.join('','C:\\Users\\Chike\\Documents\\Projects\\twitter-disaster-prediction\\models\\word_vector_model.txt'),encoding='utf-8')
# word2vec=Word2Vec.load()
for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:])
    embeddings_index[word]=coefs
f.close()

In [13]:
embeddings_index

{'2567': array(['100'], dtype='<U3'),
 'like': array(['-0.22178586', '0.023161057', '-0.619975', '0.23900992',
        '-0.6099994', '0.36013064', '-0.5591627', '0.03970661',
        '0.08087427', '0.48607442', '-0.31933376', '-0.70399',
        '-0.5597022', '0.07808574', '0.12920468', '-0.31511992',
        '-0.08946677', '-0.19172111', '-0.05379703', '0.18856083',
        '-0.018242555', '-0.46195775', '0.19191502', '0.36497363',
        '0.304077', '-0.41871834', '0.2041421', '-0.3065822', '-0.2080997',
        '0.03391011', '0.13567218', '0.1559534', '0.18585178',
        '0.26984438', '-0.08355317', '-0.4658314', '-0.15548494',
        '-0.2326282', '0.15627557', '0.12717834', '0.96179795',
        '-0.09931663', '0.08009037', '0.8613292', '-0.17148204',
        '0.48055005', '-0.0015884293', '0.60810155', '0.3060709',
        '-0.45864505', '-0.35594174', '-0.013567644', '0.13582827',
        '0.6020185', '-0.23055038', '-0.1922714', '-0.4893168',
        '-0.098422356', '0.0904

In [14]:
train_df=pd.read_csv("C:\\Users\\Chike\\Documents\\Projects\\twitter-disaster-prediction\\data\\raw\\train.csv")
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [15]:
train_df['clean_text']=train_df['text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
train_df['clean_text']=train_df['clean_text'].apply(lambda x: re.split('http:\/\/.*', str(x))[0])
train_df['clean_text']=train_df['clean_text'].str.replace('\d+', '')
train_df['clean_text']=train_df['clean_text'].str.lower()
train_df['clean_text']=train_df['clean_text'].str.replace('\W', ' ')
train_df['clean_text']=[' '.join([w for w in x.lower().split() if w not in stop]) for x in train_df['clean_text'].tolist()]
train_df['clean_text']=train_df['clean_text'].str.replace('\d+', '')

train_df.clean_text

<ipython-input-15-b6908ce19e96>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  train_df['clean_text']=train_df['clean_text'].str.replace('\d+', '')
<ipython-input-15-b6908ce19e96>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  train_df['clean_text']=train_df['clean_text'].str.replace('\W', ' ')
<ipython-input-15-b6908ce19e96>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  train_df['clean_text']=train_df['clean_text'].str.replace('\d+', '')


0            deeds reason earthquake may allah forgive us
1                   forest fire near la ronge sask canada
2       residents asked shelter place notified officer...
3       people receive wildfires evacuation orders cal...
4       got sent photo ruby alaska smoke wildfires pou...
                              ...                        
7608    two giant cranes holding bridge collapse nearb...
7609    aria_ahrary thetawniest control wild fires cal...
7610                                utc km volcano hawaii
7611    police investigating e bike collided car littl...
7612    latest homes razed northern california wildfir...
Name: clean_text, Length: 7613, dtype: object

In [16]:
 X_train, X_test, y_train, y_test = train_test_split(train_df.clean_text, train_df.target, test_size=0.2, random_state=42)

In [17]:
tokenizer=Tokenizer()
total_tweets = pd.concat([X_train, X_test])
tokenizer.fit_on_texts(total_tweets)

max_length=max([len(s.split()) for s in total_tweets])

vocab_size=len(tokenizer.word_index)+1

X_train_tokens=tokenizer.texts_to_sequences(X_train)
X_test_tokens=tokenizer.texts_to_sequences(X_test)

X_train_pad=pad_sequences(X_train_tokens,maxlen=max_length,padding='post')
X_test_pad=pad_sequences(X_test_tokens,maxlen=max_length,padding='post')

In [24]:
embeddings_index

{'2567': array(['100'], dtype='<U3'),
 'like': array(['-0.22178586', '0.023161057', '-0.619975', '0.23900992',
        '-0.6099994', '0.36013064', '-0.5591627', '0.03970661',
        '0.08087427', '0.48607442', '-0.31933376', '-0.70399',
        '-0.5597022', '0.07808574', '0.12920468', '-0.31511992',
        '-0.08946677', '-0.19172111', '-0.05379703', '0.18856083',
        '-0.018242555', '-0.46195775', '0.19191502', '0.36497363',
        '0.304077', '-0.41871834', '0.2041421', '-0.3065822', '-0.2080997',
        '0.03391011', '0.13567218', '0.1559534', '0.18585178',
        '0.26984438', '-0.08355317', '-0.4658314', '-0.15548494',
        '-0.2326282', '0.15627557', '0.12717834', '0.96179795',
        '-0.09931663', '0.08009037', '0.8613292', '-0.17148204',
        '0.48055005', '-0.0015884293', '0.60810155', '0.3060709',
        '-0.45864505', '-0.35594174', '-0.013567644', '0.13582827',
        '0.6020185', '-0.23055038', '-0.1922714', '-0.4893168',
        '-0.098422356', '0.0904

In [28]:
len(embeddings_index[next(iter(embeddings_index))])

1

In [31]:
EMBEDDING_DIM=100

In [33]:
num_words=len(tokenizer.word_index)+1
embedding_matrix=np.zeros((num_words,EMBEDDING_DIM))

for word,i in tokenizer.word_index.items():
    if i>num_words:
        continue
    embedding_vector=embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i]=embedding_vector
print(num_words)

15604


In [34]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU
from keras.layers.embeddings import Embedding
from keras.initializers import Constant

In [35]:
max_length

25

In [36]:
from tensorflow.keras.metrics import Recall
from tensorflow.keras.metrics import Precision

In [37]:
model=Sequential()
embedding_layer=Embedding(num_words
                    ,EMBEDDING_DIM
                    ,embeddings_initializer=Constant(embedding_matrix)
                    ,input_length=max_length
                    ,trainable=False)
model.add(embedding_layer)
model.add(GRU(units=32,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation="sigmoid"))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy',Precision(),Recall()])

In [39]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 100)           1560400   
_________________________________________________________________
gru (GRU)                    (None, 32)                12864     
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 1,573,297
Trainable params: 12,897
Non-trainable params: 1,560,400
_________________________________________________________________


In [40]:
model.fit(X_train_pad,y_train,batch_size=128,epochs=120,validation_data=(X_test_pad,y_test),verbose=2)

Epoch 1/120
48/48 - 29s - loss: 0.6862 - accuracy: 0.5665 - precision: 0.3678 - recall: 0.0446 - val_loss: 0.6825 - val_accuracy: 0.5739 - val_precision: 0.3750 - val_recall: 0.0091
Epoch 2/120
48/48 - 1s - loss: 0.6840 - accuracy: 0.5700 - precision: 0.3856 - recall: 0.0063 - val_loss: 0.6823 - val_accuracy: 0.5732 - val_precision: 0.3996 - val_recall: 0.0048
Epoch 3/120
48/48 - 1s - loss: 0.6834 - accuracy: 0.5698 - precision: 0.4014 - recall: 0.0040 - val_loss: 0.6819 - val_accuracy: 0.5732 - val_precision: 0.4121 - val_recall: 0.0035
Epoch 4/120
48/48 - 1s - loss: 0.6804 - accuracy: 0.5732 - precision: 0.4150 - recall: 0.0040 - val_loss: 0.6740 - val_accuracy: 0.6080 - val_precision: 0.5255 - val_recall: 0.0294
Epoch 5/120
48/48 - 1s - loss: 0.6678 - accuracy: 0.5839 - precision: 0.5263 - recall: 0.0656 - val_loss: 0.6548 - val_accuracy: 0.6139 - val_precision: 0.5308 - val_recall: 0.0981
Epoch 6/120
48/48 - 1s - loss: 0.6611 - accuracy: 0.5956 - precision: 0.5320 - recall: 0.1272 

48/48 - 1s - loss: 0.6494 - accuracy: 0.6189 - precision: 0.5710 - recall: 0.3908 - val_loss: 0.6394 - val_accuracy: 0.6310 - val_precision: 0.5712 - val_recall: 0.3904
Epoch 47/120
48/48 - 1s - loss: 0.6482 - accuracy: 0.6235 - precision: 0.5715 - recall: 0.3902 - val_loss: 0.6392 - val_accuracy: 0.6323 - val_precision: 0.5718 - val_recall: 0.3902
Epoch 48/120
48/48 - 1s - loss: 0.6492 - accuracy: 0.6190 - precision: 0.5720 - recall: 0.3909 - val_loss: 0.6414 - val_accuracy: 0.6316 - val_precision: 0.5721 - val_recall: 0.3906
Epoch 49/120
48/48 - 1s - loss: 0.6488 - accuracy: 0.6184 - precision: 0.5723 - recall: 0.3904 - val_loss: 0.6377 - val_accuracy: 0.6389 - val_precision: 0.5725 - val_recall: 0.3905
Epoch 50/120
48/48 - 1s - loss: 0.6476 - accuracy: 0.6282 - precision: 0.5729 - recall: 0.3903 - val_loss: 0.6390 - val_accuracy: 0.6336 - val_precision: 0.5732 - val_recall: 0.3903
Epoch 51/120
48/48 - 1s - loss: 0.6469 - accuracy: 0.6245 - precision: 0.5735 - recall: 0.3902 - val_lo

Epoch 92/120
48/48 - 1s - loss: 0.6399 - accuracy: 0.6343 - precision: 0.5898 - recall: 0.3888 - val_loss: 0.6328 - val_accuracy: 0.6520 - val_precision: 0.5900 - val_recall: 0.3889
Epoch 93/120
48/48 - 1s - loss: 0.6425 - accuracy: 0.6338 - precision: 0.5902 - recall: 0.3889 - val_loss: 0.6293 - val_accuracy: 0.6408 - val_precision: 0.5903 - val_recall: 0.3889
Epoch 94/120
48/48 - 1s - loss: 0.6411 - accuracy: 0.6330 - precision: 0.5905 - recall: 0.3889 - val_loss: 0.6283 - val_accuracy: 0.6533 - val_precision: 0.5907 - val_recall: 0.3888
Epoch 95/120
48/48 - 1s - loss: 0.6397 - accuracy: 0.6333 - precision: 0.5909 - recall: 0.3890 - val_loss: 0.6310 - val_accuracy: 0.6546 - val_precision: 0.5910 - val_recall: 0.3890
Epoch 96/120
48/48 - 1s - loss: 0.6396 - accuracy: 0.6322 - precision: 0.5910 - recall: 0.3891 - val_loss: 0.6275 - val_accuracy: 0.6461 - val_precision: 0.5912 - val_recall: 0.3891
Epoch 97/120
48/48 - 1s - loss: 0.6378 - accuracy: 0.6300 - precision: 0.5913 - recall: 0.

In [41]:
val_df=pd.read_csv("C:\\Users\\Chike\\Documents\\Projects\\twitter-disaster-prediction\\data\\raw\\test.csv")
val_df

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [42]:
val_df['clean_text']=val_df['text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
val_df['clean_text']=val_df['clean_text'].apply(lambda x: re.split('http:\/\/.*', str(x))[0])
val_df['clean_text']=val_df['clean_text'].str.replace('\d+', '')
val_df['clean_text']=val_df['clean_text'].str.lower()
val_df['clean_text']=val_df['clean_text'].str.replace('\W', ' ')
val_df['clean_text']=[' '.join([w for w in x.lower().split() if w not in stop]) for x in val_df['clean_text'].tolist()]
val_df['clean_text']=val_df['clean_text'].str.replace('\d+', '')

val_df.clean_text

<ipython-input-42-99147990c7c3>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  val_df['clean_text']=val_df['clean_text'].str.replace('\d+', '')
<ipython-input-42-99147990c7c3>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  val_df['clean_text']=val_df['clean_text'].str.replace('\W', ' ')
<ipython-input-42-99147990c7c3>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  val_df['clean_text']=val_df['clean_text'].str.replace('\d+', '')


0                             happened terrible car crash
1       heard earthquake different cities stay safe ev...
2       forest fire spot pond geese fleeing across str...
3                   apocalypse lighting spokane wildfires
4                     typhoon soudelor kills china taiwan
                              ...                        
3258    earthquake safety los angeles ûò safety fasten...
3259    storm ri worse last hurricane city amp others ...
3260                        green line derailment chicago
3261             meg issues hazardous weather outlook hwo
3262    cityofcalgary activated municipal emergency pl...
Name: clean_text, Length: 3263, dtype: object

In [46]:
def predictions(text):
    sequence=tokenizer.texts_to_sequences(text)
        # pad the sequences
    sequence = pad_sequences(sequence, maxlen=max_length,padding='post')
        # get the prediction
    prediction = model.predict(sequence)[0]
    return prediction[0]#, [np.argmax(prediction)]

In [43]:
val_tokens=tokenizer.texts_to_sequences(val_df.clean_text)

val_pad=pad_sequences(val_tokens,maxlen=max_length,padding='post')

In [44]:
val_df['predictions']=model.predict(val_pad)

In [45]:
val_df.sort_values('predictions',ascending=False)

,id,keyword,location,text,clean_text,predictions
2928,9693,tornado,"riverwest, milw",@dead_last I'm going to live out of my car for...,dead_last going live car year probably life goals,0.933853
1195,3930,devastated,Dhaka,Obama Declares Disaster for Typhoon-Devastated...,obama declares disaster typhoon devastated sai...,0.929803
3042,10063,typhoon,ID,Obama Declares Disaster for Typhoon-Devastated...,obama declares disaster typhoon devastated sai...,0.929803
1189,3915,devastated,NaN,Obama Declares Disaster for Typhoon-Devastated...,obama declares disaster typhoon devastated sai...,0.929803
1191,3920,devastated,NaN,Obama Declares Disaster for Typhoon-Devastated...,obama declares disaster typhoon devastated sai...,0.929803
...,...,...,...,...,...,...
1800,6080,heat%20wave,NaN,http://t.co/OYMLNLHEca So important to stay hy...,,0.231476
838,2750,crushed,NaN,http://t.co/ACHBbiFQrQ : Big Papi with a great...,,0.231476
295,966,blaze,NaN,Craving slurpees ;-;,craving slurpees,0.231476
2481,8290,rioting,NaN,http://t.co/DYKYpy3vA5 Cindy Noonan@CindyNoon...,,0.231476


In [46]:
val_df['target']=np.where(val_df['predictions']>=0.5,1,0)

In [47]:
val_df.target.value_counts()

0    2191
1    1072
Name: target, dtype: int64

In [48]:
submission_df=val_df[['id','target']]
submission_df

,id,target
0,0,0
1,2,1
2,3,0
3,9,0
4,11,0
...,...,...
3258,10861,0
3259,10865,0
3260,10868,0
3261,10874,0


In [49]:
submission_df.to_csv("C:\\Users\\Chike\\Documents\\Projects\\twitter-disaster-prediction\\data\\processed\\2.1_tensorflow_gensim_word2vec.csv",index=False)

In [80]:
!pwd

/c/Users/Chike/Documents/Projects/twitter-disaster-prediction/notebooks
